In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'sml-project:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F50088%2F5285395%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240707%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240707T081749Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D049d5d662f1f34c12f2e7e6c275bd4fcd27f89d2699fb3cfccf94310af06c0491d31bfd1583c08a6999311fe45878e2d6b81fa0c75e9f2af51a2da0492beed7cbc678ef74c46f0f21b94ba9d43ded56db9d9eee210f857d44f6259ada566920c98b55223a4f7406141945779082ebf8a34a626f838c4d3a6765bc15cbf3d6002b660b90cea9fcaba9c06c3fd9e5529ea434060ee053a4189c59e956504b9f91c7961cb636e32989c007abae6539f27003d279929831ef711f9aa29275afdb325ad55de6641facb640bd463c94834cb64adc320ccda902e755b5698fb4aa4294fa095237fa9ce5356dde3030ccbd9ca8bc1828eeaeb1cbfecf7aed98c4d561b0e'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
training_data = pd.read_csv("/kaggle/input/sml-project/train.csv")
new_x = training_data
training_data.dropna
training_data.drop_duplicates
training_data = training_data.drop('category', axis=1)
training_data=training_data.drop('ID',axis=1)

same_values_mask = (training_data == training_data.iloc[0]).all(axis=0)
training_data = training_data.loc[:, ~same_values_mask]

# zero_mask = (training_data == 0.0).all(axis=0)
# training_data = training_data.loc[:, ~zero_mask]

nonzero_mask = (training_data != 0).sum(axis=0) < 25
training_data = training_data.loc[:, ~nonzero_mask]
training_data.head()

In [ ]:
testing_data = pd.read_csv("/kaggle/input/sml-project/test.csv")
new_y = testing_data
testing_data.dropna
testing_data.drop_duplicates
testing_data=testing_data.drop('ID',axis=1)

testing_data = testing_data.loc[:, ~same_values_mask]
testing_data = testing_data.loc[:, ~nonzero_mask]

testing_data.head()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import LocalOutlierFactor
from sklearn.ensemble import IsolationForest
from sklearn.feature_selection import RFE

import warnings
warnings.filterwarnings('ignore')

Y_output = new_x["category"]
X_modelfitting = training_data
X_testing = testing_data

In [ ]:
isolationtree = IsolationForest()
outliers = isolationtree.fit_predict(X_modelfitting)
number_outliers = len(outliers[outliers == -1])
print("Number of outliers:", number_outliers)
mask = outliers != -1
X_modelfitting = X_modelfitting[mask]
Y_output = Y_output[mask]

In [ ]:
pca_to_best_components = PCA(n_components=1000)
X_modelfitting = pca_to_best_components.fit_transform(X_modelfitting)
X_testing = pca_to_best_components.transform(X_testing)

In [ ]:
# estimator_using_rfe = LogisticRegression(max_iter=10000)
# selector_using_rfe = RFE(estimator_using_rfe, n_features_to_select=470, step=1)
# selector_using_rfe.fit(X_modelfitting, Y_output)
# X_modelfitting = X_modelfitting[:,selector_using_rfe.support_]
# X_testing = X_testing[:,selector_using_rfe.support_]
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

import warnings
warnings.filterwarnings('ignore')

# pca_to_best_components = PCA(n_components=400)
# X_modelfitting = pca_to_best_components.fit_transform(X_modelfitting)
# X_testing = pca_to_best_components.transform(X_testing)

In [ ]:
for i in range(3):
    X_modelfitting_df = pd.DataFrame(data=X_modelfitting)
    # Apply KMeans Clustering for grouping similar samples together on X_modelfitting
    cluster_model = KMeans(n_clusters=20)
    cluster_assignments_train = cluster_model.fit_predict(X_modelfitting)

    # Add the cluster assignments as a new column in X_modelfitting
    X_modelfitting_df = pd.DataFrame(data=X_modelfitting)
    X_modelfitting_df[f'cluster{i}'] = cluster_assignments_train

    # Create a new feature based on the cluster assignments
    cluster_counts = X_modelfitting_df[f'cluster{i}'].value_counts(normalize=True)
    X_modelfitting_df[f'cluster_ratio{i}'] = X_modelfitting_df[f'cluster{i}'].map(cluster_counts)

    # Transform X_testing using the cluster model fit on X_modelfitting
    X_testing_df = pd.DataFrame(data=X_testing)
    cluster_assignments_test = cluster_model.predict(X_testing_df)
    X_testing_df[f'cluster{i}'] = cluster_assignments_test
    X_testing_df[f'cluster_ratio{i}'] = X_testing_df[f'cluster{i}'].map(cluster_counts)

    # Select only the cluster id and cluster ratio columns from train and test sets
    X_modelfitting_cluster = X_modelfitting_df[[f'cluster{i}', f'cluster_ratio{i}']].to_numpy()
    X_testing_cluster =X_testing_df[[f'cluster{i}', f'cluster_ratio{i}']].to_numpy()

In [ ]:
pca_to_best_components = PCA(n_components=400)
X_modelfitting = pca_to_best_components.fit_transform(X_modelfitting)
X_testing = pca_to_best_components.transform(X_testing)

linear_analysis = LinearDiscriminantAnalysis(n_components = 19)
X_train_lda = linear_analysis.fit_transform(X_modelfitting, Y_output)
X_test_lda = linear_analysis.transform(X_testing)

In [ ]:
# Concatenate X_train_cluster and X_train_final
X_train_final = np.concatenate((X_modelfitting_cluster, X_train_lda), axis=1)

# Concatenate X_test_cluster and X_test_final
X_test_final = np.concatenate((X_testing_cluster, X_test_lda), axis=1)
scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train_final)
# X_test_scaled = scaler.transform(X_test_final)

In [ ]:
# logistic = LogisticRegression(max_iter=10000,C=0.1)
# logistic.fit(X_train_final, Y_output)
# Y_predicted = logistic.predict(X_test_final)
# print(Y_predicted)

# Train a neural network
neural_net = MLPClassifier(hidden_layer_sizes=(300,), max_iter=800)
bagging = BaggingClassifier(neural_net, n_estimators=300)

# Fit the bagging classifier to the training data
bagging.fit(X_train_final, Y_output)

# Predict on the test data
Y_predicted = bagging.predict(X_test_final)
print(Y_predicted)

In [ ]:
output = pd.DataFrame({'ID': new_y.ID, 'category': Y_predicted})
output.to_csv('submission.csv', index=False)